In [1]:
import pandas as pd


In [2]:
ratings_df = pd.read_excel('DATA\clean_rating3.xlsx')

In [3]:
ratings_df.columns

Index(['userID', 'Book-Rating', 'Publication Year', 'num_of_rating',
       'ISBN_encoded', 'Title_encoded', 'author_encoded', 'image_encoded',
       'publisher_encoded'],
      dtype='object')

In [4]:

ratings_df.rename(columns={'ISBN_encoded': 'ISBN'}, inplace=True)
ratings_df.rename(columns={'Title_encoded': 'Title'}, inplace=True)
ratings_df.rename(columns={'iamge_encoded': 'image'}, inplace=True)
ratings_df.rename(columns={'publisher_encoded': 'publisher'}, inplace=True)
ratings_df.rename(columns={'author_encoded': 'author'}, inplace=True)



In [5]:

book_names =ratings_df.set_index('ISBN')['Title'].to_dict()
n_users = len(ratings_df.userID.unique())
n_items = len(ratings_df.ISBN.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")
     

Number of unique users: 887
Number of unique movies: 1836
The full rating matrix will have: 1628532 elements.
----------
Number of ratings: 51757
Therefore:  3.178138347910879 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [6]:

import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [7]:
ratings_df.head(2)


,userID,Book-Rating,Publication Year,num_of_rating,ISBN,Title,author,image_encoded,publisher
0,277427,0,1999,133,0,0,0,132,51294
1,254,0,1999,133,0,0,0,132,51294


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

class Loader(Dataset):
    def __init__(self, ratings_df):
        self.ratings = ratings_df.copy()
        #self.books = ratings_df.copy()
        
        # Extract all user IDs and book ISBNs
        users = ratings_df.userID.unique()
        isbns = ratings_df.ISBN.unique()
        
        # Producing new continuous IDs for users and books
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.isbn2idx = {o:i for i,o in enumerate(isbns)}
        
        # Obtained continuous ID for users and books
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2isbn = {i:o for o,i in self.isbn2idx.items()}
        
        # Map the original user and book IDs to the continuous IDs
        self.ratings.ISBN = ratings_df.ISBN.apply(lambda x: self.isbn2idx[x])
        self.ratings.userID = ratings_df.userID.apply(lambda x: self.userid2idx[x])
        
        # Prepare the input features (x) and target labels (y) for the model
        self.x = self.ratings.drop(['Book-Rating','author'], axis=1).values
        self.y = self.ratings['Book-Rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)


In [11]:

import torch


num_epochs = 200


model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
loss_fn = torch.nn.MSELoss()


# Train data
train_set = Loader(ratings_df)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)



MatrixFactorization(
  (user_factors): Embedding(887, 8)
  (item_factors): Embedding(1836, 8)
)
user_factors.weight tensor([[0.0133, 0.0072, 0.0050,  ..., 0.0280, 0.0163, 0.0266],
        [0.0210, 0.0432, 0.0094,  ..., 0.0315, 0.0344, 0.0380],
        [0.0317, 0.0206, 0.0029,  ..., 0.0179, 0.0156, 0.0459],
        ...,
        [0.0133, 0.0305, 0.0364,  ..., 0.0118, 0.0325, 0.0332],
        [0.0331, 0.0422, 0.0170,  ..., 0.0062, 0.0203, 0.0276],
        [0.0176, 0.0351, 0.0253,  ..., 0.0472, 0.0261, 0.0341]])
item_factors.weight tensor([[0.0227, 0.0214, 0.0151,  ..., 0.0074, 0.0297, 0.0342],
        [0.0375, 0.0284, 0.0252,  ..., 0.0495, 0.0085, 0.0008],
        [0.0008, 0.0294, 0.0418,  ..., 0.0036, 0.0093, 0.0397],
        ...,
        [0.0323, 0.0196, 0.0140,  ..., 0.0066, 0.0378, 0.0083],
        [0.0252, 0.0474, 0.0271,  ..., 0.0454, 0.0214, 0.0494],
        [0.0213, 0.0262, 0.0259,  ..., 0.0135, 0.0171, 0.0030]])


In [12]:
import tqdm
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # Define the optimizer
optimizer.zero_grad()  # Clear the gradients
for epoch in tqdm.tqdm(range(num_epochs)):

    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        
        # Ensure that the input data is within the valid range of indices
        batch_x = torch.clamp(batch_x, 0, n_users-1)
        
        outputs = model(batch_x)
        loss = loss_fn(outputs, batch_y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")



  0%|          | 1/200 [00:00<02:41,  1.23it/s]

Epoch 1, Loss: 15.985831829353614


  1%|          | 2/200 [00:01<02:41,  1.23it/s]

Epoch 2, Loss: 15.053939072879745


  2%|▏         | 3/200 [00:02<02:41,  1.22it/s]

...

Epoch 198, Loss: 10.10224911842817


100%|█████████▉| 199/200 [03:39<00:01,  1.09s/it]

Epoch 199, Loss: 10.097396389054664


100%|██████████| 200/200 [03:40<00:00,  1.10s/it]

Epoch 200, Loss: 10.099863451498527
